In [4]:
def isEvenBitwise(value):
  return (value & 1) == 0

def isEven(value):
  return value % 2 == 0

def test_evenness(value):
  print(f"isEven: {isEven(value)}")
  print(f"isEvenBitwise: {isEvenBitwise(value)}\n")

test_evenness(10)
test_evenness(11)

isEven: True
isEvenBitwise: True

isEven: False
isEvenBitwise: False



# Использование модуля (%):

## Плюсы:

- Понятно всем: Этот способ проверки четности - самый распространенный и большинство людей, даже не очень знакомых с программированием, быстро поймут, что происходит.
- Работает со всеми числами: Не важно, положительное это число или отрицательное - этот метод всегда сработает.

## Минусы:

- Может быть медленнее: Нахождение остатка от деления может занять больше времени, особенно если числа очень большие.

# Использование побитового 'И' (AND):

## Плюсы:

- Быстрее: Этот способ часто работает быстрее, особенно если речь идет о больших числах.

## Минусы:

- Не для всех понятно: Если вы не знакомы с побитовыми операциями, этот код может показаться загадочным.
- Только для целых чисел: Этот метод подходит только для целых чисел.

In [7]:
from collections import deque
import time

class CircularBufferList:
    def __init__(self, capacity):
        self.buffer = [None] * capacity
        self.capacity = capacity
        self.start = 0
        self.end = 0
        self.size = 0

    def enqueue(self, item):
        if self.size == self.capacity:
            raise Exception("Буфер заполнен")
        self.buffer[self.end] = item
        self.end = (self.end + 1) % self.capacity
        self.size += 1

    def dequeue(self):
        if self.size == 0:
            raise Exception("Буфер пуст")
        item = self.buffer[self.start]
        self.start = (self.start + 1) % self.capacity
        self.size -= 1
        return item

    def __str__(self):
        return str(self.buffer[self.start:self.end + 1])

class CircularBufferDeque:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def enqueue(self, item):
        if len(self.buffer) == self.buffer.maxlen:
            self.dequeue()
        self.buffer.append(item)

    def dequeue(self):
        if not self.buffer:
            raise Exception("Буфер пуст")
        return self.buffer.popleft()

    def __str__(self):
        return str(list(self.buffer))

def test_circular_buffer(buffer_class, capacity):
    print(f"Тестируем {buffer_class.__name__} с вместимостью {capacity}")
    buffer = buffer_class(capacity)

    start_time = time.time()
    for i in range(capacity):
        buffer.enqueue(i)
    enqueue_time = time.time() - start_time
    print(f"Время на добавление элементов: {enqueue_time:.6f} секунд")

    start_time = time.time()
    removed_item = buffer.dequeue()
    dequeue_time = time.time() - start_time
    print(f"Время на удаление элемента: {dequeue_time:.6f} секунд")

    print("Удаленный элемент:", removed_item)
    print("Содержимое буфера после удаления элемента:", buffer)
    print("")

test_circular_buffer(CircularBufferList, 5)
test_circular_buffer(CircularBufferDeque, 5)

Тестируем CircularBufferList с вместимостью 5
Время на добавление элементов: 0.000010 секунд
Время на удаление элемента: 0.000006 секунд
Удаленный элемент: 0
Содержимое буфера после удаления элемента: []

Тестируем CircularBufferDeque с вместимостью 5
Время на добавление элементов: 0.000008 секунд
Время на удаление элемента: 0.000005 секунд
Удаленный элемент: 0
Содержимое буфера после удаления элемента: [1, 2, 3, 4]



- Обе реализации достаточно эффективны для небольшого количества элементов, что демонстрируют результаты тестирования.

- CircularBufferDeque демонстрирует незначительное преимущество в скорости по сравнению с CircularBufferList. Это может быть более заметно при работе с большим количеством элементов или в более требовательных по производительности сценариях.

- Реализация на основе deque предпочтительнее для ситуаций, где требуется высокая производительность операций вставки и удаления.

In [8]:
import time
import random
from collections import deque

def calc_min_run(n):
    r = 0
    while n >= 64:
        r |= n & 1
        n >>= 1
    return n + r

def insertion_sort(arr, left, right):
    for i in range(left + 1, right + 1):
        temp = arr[i]
        j = i - 1
        while j >= left and arr[j] > temp:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = temp

def merge(arr, l, m, r):
    len1, len2 = m - l + 1, r - m
    left, right = [], []
    for i in range(0, len1):
        left.append(arr[l + i])
    for i in range(0, len2):
        right.append(arr[m + 1 + i])

    i, j, k = 0, 0, l

    while i < len1 and j < len2:
        if left[i] <= right[j]:
            arr[k] = left[i]
            i += 1
        else:
            arr[k] = right[j]
            j += 1
        k += 1

    while i < len1:
        arr[k] = left[i]
        i += 1
        k += 1

    while j < len2:
        arr[k] = right[j]
        j += 1
        k += 1

def tim_sort(arr):
    n = len(arr)
    min_run = calc_min_run(n)

    # Сортировка вставками для каждого подмассива размером min_run
    for start in range(0, n, min_run):
        end = min(start + min_run - 1, n - 1)
        insertion_sort(arr, start, end)

    # Слияние отсортированных подмассивов
    size = min_run
    while size < n:
        for left in range(0, n, 2 * size):
            mid = min(n - 1, left + size - 1)
            right = min((left + 2 * size - 1), (n - 1))

            if mid < right:
                merge(arr, left, mid, right)

        size *= 2

# Стандартная сортировка Python
def sort_array(arr):
    return sorted(arr)

# Функция для измерения времени выполнения сортировки
def measure_time(func, arr):
    start_time = time.time()
    func(arr)
    return time.time() - start_time

# Генерация случайного массива
array = [random.randint(0, 1000) for _ in range(1000)]

# Копия массива для справедливого сравнения
array_copy = array.copy()

# Измерение времени для встроенной сортировки
time_sorted = measure_time(sort_array, array)
print(f"Время, затраченное на стандартную сортировку: {time_sorted:.6f} секунд")

# Измерение времени для Timsort
time_timsort = measure_time(tim_sort, array_copy)
print(f"Время, затраченное на Timsort: {time_timsort:.6f} секунд")

Время, затраченное на стандартную сортировку: 0.000137 секунд
Время, затраченное на Timsort: 0.004654 секунд
